# Data preprocessing

In [1]:
# Import libraries
import pandas as pd
import os
import csv
import json
import billboard
import re
import lyricsgenius
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from langdetect import detect
from pandas_profiling import ProfileReport
pd.options.mode.chained_assignment = None

### Genius and Spotify APIs' settings

In [2]:
# Genius token
genius_token = os.environ.get('genius_token')

# Spotify authorization
spotify_client_id = os.environ.get('spotify_client_id')
spotify_client_secret = os.environ.get('spotify_client_secret')
client_credentials_manager = SpotifyClientCredentials(client_id = spotify_client_id, client_secret = spotify_client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

### CSV data

In [3]:
# Loading data and adding year columns and wrapping up billboard songs (1950-2015)
wd = os.getcwd()
data_path = os.chdir(wd + '\data')
all_csv_files = os.listdir(data_path)
songs_50_15 = []

for filename in all_csv_files:
    songs = pd.read_csv(filename, header = 0, sep = ';')
    year = os.path.basename(filename)[:-4]
    songs['Year'] = year
    songs_50_15.append(songs)

### billboard library data

In [4]:
# Downloading data from years 2016-2021 from billboard library
playlist = 'hot-100-songs'
dates = [2016, 2017, 2018, 2019, 2020, 2021]
songs_billboard = []
for y in dates:
    songs = billboard.ChartData(playlist, date = None, year = y, fetch = True, timeout = 25)
    songs_billboard.append(songs)

In [5]:
# Preprocessing data from years 2016-2021
song_titles, artists = [[] for i in range(2)]

for year in songs_billboard:
    for playlist in year:
        song = str(playlist).split(' by ')
        song_titles.append(song[0].strip("\'"))
        artists.append(song[1])
        
# Adding missed song on 87th position in 2016 chart
song_titles.insert(86, 'All the Way Up')
artists.insert(86, 'Fat Joe, Remy Ma and Jay-Z featuring French Montana and Infered')

### Concatenating CSV and billboard library data

In [6]:
# Splitting list into same-length elements lists
def list_split(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

split_artists = list(list_split(artists, 100))
split_song_titles = list(list_split(song_titles, 100))

# Creating list of dictionaries for each year to convert them into pandas DataFrame
def dicts_of_dates(lst_of_dates):
    songs = []
    for i in range(len(lst_of_dates)):
        ranger = list(range(1, 101))
        year = lst_of_dates[i]
        dictionary = {'Position': ranger, 'Artist': split_artists[i], 'Song Title': split_song_titles[i], 'Year': [dates[i] for year in range(1, 101)]}
        as_pandas_df = pd.DataFrame(dictionary)
        songs.append(as_pandas_df)
    return songs

# Concatenate all songs from years 1950 - 2021
songs_16_21 = dicts_of_dates(dates)
all_songs = songs_50_15 + songs_16_21
all_time_billboard_wrap_up = pd.concat(all_songs)
all_time_billboard_wrap_up.reset_index(drop = True, inplace = True) # reset indexing
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950
1,2,Nat King Cole,Mona Lisa,1950
2,3,Anton Karas,Third Man Theme,1950
3,4,Gary and Bing Crosby,Sam's Song,1950
4,5,Gary and Bing Crosby,Simple Melody,1950
...,...,...,...,...
6775,96,Lainey Wilson,Things A Man Oughta Know,2021
6776,97,BRS Kash,Throat Baby (Go Baby),2021
6777,98,Rod Wave,Tombstone,2021
6778,99,Chase Rice Featuring Florida Georgia Line,Drinkin' Beer. Talkin' God. Amen.,2021


In [7]:
# Creating columns for json files' names
all_time_billboard_wrap_up['JSON file'] = [str(i) + '.json' for i in all_time_billboard_wrap_up.index]
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year,JSON file
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,0.json
1,2,Nat King Cole,Mona Lisa,1950,1.json
2,3,Anton Karas,Third Man Theme,1950,2.json
3,4,Gary and Bing Crosby,Sam's Song,1950,3.json
4,5,Gary and Bing Crosby,Simple Melody,1950,4.json
...,...,...,...,...,...
6775,96,Lainey Wilson,Things A Man Oughta Know,2021,6775.json
6776,97,BRS Kash,Throat Baby (Go Baby),2021,6776.json
6777,98,Rod Wave,Tombstone,2021,6777.json
6778,99,Chase Rice Featuring Florida Georgia Line,Drinkin' Beer. Talkin' God. Amen.,2021,6778.json


In [8]:
# Spliting authors by 'and', 'featuring', 'feat.', Featuring', 'Feat.', '&' to make api searching easier and more efficient
artists = all_time_billboard_wrap_up['Artist'].to_list()
split_artists = []
for artist in artists:
    if any(re.findall(r'and|featuring|feat.|Featuring|Feat.|&', str(artist))):
        result = re.split(r'and|featuring|feat.|Featuring|Feat.|&', str(artist))[0].strip()
        split_artists.append(result)
    else:
        split_artists.append(artist)

all_time_billboard_wrap_up['Split Names'] = split_artists
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year,JSON file,Split Names
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,0.json,Gordon Jenkins
1,2,Nat King Cole,Mona Lisa,1950,1.json,Nat King Cole
2,3,Anton Karas,Third Man Theme,1950,2.json,Anton Karas
3,4,Gary and Bing Crosby,Sam's Song,1950,3.json,Gary
4,5,Gary and Bing Crosby,Simple Melody,1950,4.json,Gary
...,...,...,...,...,...,...
6775,96,Lainey Wilson,Things A Man Oughta Know,2021,6775.json,Lainey Wilson
6776,97,BRS Kash,Throat Baby (Go Baby),2021,6776.json,BRS Kash
6777,98,Rod Wave,Tombstone,2021,6777.json,Rod Wave
6778,99,Chase Rice Featuring Florida Georgia Line,Drinkin' Beer. Talkin' God. Amen.,2021,6778.json,Chase Rice


### Genius data into json files

In [9]:
# lyricsgenius settings
genius = lyricsgenius.Genius(genius_token, timeout = 30, retries = 3)
jsons_path = wd + '\\jsons\\'

# Saving lyrics to jsons function
def return_lyrics_json(song, artist, json_name):
    song_lyrics = genius.search_song(song, artist)
    with open(jsons_path + json_name, 'w', encoding = 'UTF-8') as f:
        json.dump({'lyrics': song_lyrics.lyrics}, f, ensure_ascii = False, indent = 4)

# No lyrics function (errors or instrumental songs)
def no_lyrics(json_name):
    with open(jsons_path + json_name, 'w', encoding = 'UTF-8') as f:
        json.dump({'lyrics': ""}, f, ensure_ascii = False, indent = 4)

In [10]:
# Applying return_lyrics_json and no_lyrics functions on lyrics:
def json_data():
    for i in range(len(all_time_billboard_wrap_up)):
        try:
            return_lyrics_json(all_time_billboard_wrap_up['Song Title'][i], all_time_billboard_wrap_up['Artist'][i], all_time_billboard_wrap_up['JSON file'][i])
        except AttributeError:
            no_lyrics(all_time_billboard_wrap_up['JSON file'][i])
json_data()

Searching for "Goodnight Irene" by Gordon Jenkins and The Weavers...
No results found for: 'Goodnight Irene Gordon Jenkins and The Weavers'
Searching for "Mona Lisa" by Nat King Cole...
Done.
Searching for "Third Man Theme" by Anton Karas...
Done.
Searching for "Sam's Song" by Gary and Bing Crosby...
Done.
Searching for "Simple Melody" by Gary and Bing Crosby...
Done.
Searching for "Music, Music, Music" by Teresa Brewer...
Done.
Searching for "Third Man Theme" by Guy Lombardo...
Done.
Searching for "Chattanoogie Shoe Shine Boy" by Red Foley...
Done.
Searching for "Harbor Lights" by Sammy Kaye...
Done.
Searching for "It Isn't Fair" by Sammy Kaye and Don Cornell...
No results found for: 'It Isn't Fair Sammy Kaye and Don Cornell'
Searching for "If I Knew You Were Coming I'd've Baked A Cake" by Eileen Barton...
Done.
Searching for "Bonaparte's Retreat" by Kay Starr...
Done.
Searching for "Tzena, Tzena, Tzena" by Gordon Jenkins and The Weavers...
No results found for: 'Tzena, Tzena, Tzena G

Done.
Searching for "C'est Si Bon" by Eartha Kitt...
Done.
Searching for "Have You Heard?" by Joni James...
Done.
Searching for "Little Things Mean A Lot" by Kitty Kallen...
Done.
Searching for "Wanted" by Perry Como...
Done.
Searching for "Hey There" by Rosemary Clooney...
Done.
Searching for "Sh-Boom" by Crew Cuts...
Done.
Searching for "Make Love To Me" by Jo Stafford...
Done.
Searching for "Oh! My Pa-pa" by Eddie Fisher...
Done.
Searching for "I Get So Lonely" by Four Knights...
Done.
Searching for "Three Coins In The Fountain" by Four Aces...
Done.
Searching for "Secret Love" by Doris Day...
Done.
Searching for "Hernando's Hideaway" by Archie Bleyer...
Done.
Searching for "Young At Heart" by Frank Sinatra...
Done.
Searching for "This Ole House" by Rosemary Clooney...
Done.
Searching for "I Need You Now" by Eddie Fisher...
Done.
Searching for "Cross Over The Bridge" by Patti Page...
Done.
Searching for "The Little Shoemaker" by Gaylords...
Done.
Searching for "That's Amore" by Dean

Done.
Searching for "Song For a Summer Night" by Mitch Miller...
Done.
Searching for "Why Do Fools Fall in Love" by Gale Storm...
Done.
Searching for "Teen Age Prayer" by Gale Storm...
Done.
Searching for "Picnic" by McGuire Sisters...
Done.
Searching for "After the Lights Go Down Low" by Al Hibbler...
Done.
Searching for "Mama From the Train" by Patti Page...
Done.
Searching for "That's All There Is to That" by Nat King Cole...
Done.
Searching for "Band of Gold" by Kit Carson...
Done.
Searching for "Walk Hand in Hand" by Tony Martin...
Done.
Searching for "Why Do Fools Fall in Love" by Diamonds...
Done.
Searching for "Juke Box Baby" by Perry Como...
Done.
Searching for "Are You Satisfied" by Rusty Draper...
Done.
Searching for "It's Almost Tomorrow" by Jo Stafford...
Done.
Searching for "You'll Never Never Know" by Platters...
Done.
Searching for "Glendora" by Perry Como...
Done.
Searching for "I Walk the Line" by Johnny Cash...
Done.
Searching for "Can You Find It in Your Heart" by T

No results found for: 'Volare (Nel Blu Dipinto Di Blu) Domenico Modugno'
Searching for "All I Have To Do Is Dream / Claudette" by Everly Brothers...
Done.
Searching for "Don't / I Beg Of You" by Elvis Presley...
Done.
Searching for "Witch Doctor" by David Seville...
Done.
Searching for "Patricia" by Perez Prado...
Done.
Searching for "Sail Along Silvery Moon / Raunchy" by Billy Vaughn...
No results found for: 'Sail Along Silvery Moon / Raunchy Billy Vaughn'
Searching for "Catch A Falling Star / Magic Moments" by Perry Como...
No results found for: 'Catch A Falling Star / Magic Moments Perry Como'
Searching for "Tequila" by Champs...
Done.
Searching for "It's All In The Game" by Tommy Edwards...
Done.
Searching for "Return To Me" by Dean Martin...
Done.
Searching for "It's Only Make Believe" by Conway Twitty...
Done.
Searching for "The Purple People Eater" by Sheb Wooley...
Done.
Searching for "Bird Dog / Devoted To You" by Everly Brothers...
Done.
Searching for "Get A Job" by Silhouett

Done.
Searching for "A Big Hunk O' Love" by Elvis Presley...
Done.
Searching for "Red River Rock" by Johnny and The Hurricanes...
Done.
Searching for "Waterloo" by Stonewall Jackson...
Done.
Searching for "Lavender Blue (Dilly Dilly)" by Sammy Turner...
Done.
Searching for "(Now And Then There's) A Fool Such As I" by Elvis Presley...
Done.
Searching for "Guitar Boogie Shuffle" by Virtues...
Done.
Searching for "Teen Beat" by Sandy Nelson...
Specified song does not contain lyrics. Rejecting.
Searching for "Kookie Kookie (Lend Me Your Comb)" by Edward Burns and Connie Stevens...
No results found for: 'Kookie Kookie (Lend Me Your Comb) Edward Burns and Connie Stevens'
Searching for "Tragedy" by Thomas Wayne...
Done.
Searching for "My Happiness" by Connie Francis...
Done.
Searching for "Talahassee Lassie" by Freddie Cannon...
No results found for: 'Talahassee Lassie Freddie Cannon'
Searching for "Tiger" by Fabian...
Done.
Searching for "Never Be Anyone Else But You" by Ricky Nelson...
Done

...

Done.
Searching for "Can't Stop The Feeling!" by Justin Timberlake...
Done.
Searching for "Closer" by The Chainsmokers Featuring Halsey...
Done.
Searching for "Cheap Thrills" by Sia Featuring Sean Paul...
Done.
Searching for "7 Years" by Lukas Graham...
Done.
Searching for "Needed Me" by Rihanna...
Done.
Searching for "My House" by Flo Rida...
Done.
Searching for "I Took A Pill In Ibiza" by Mike Posner...
Done.
Searching for "Work From Home" by Fifth Harmony Featuring Ty Dolla $ign...
Done.
Searching for "This Is What You Came For" by Calvin Harris Featuring Rihanna...
Done.
Searching for "Cake By The Ocean" by DNCE...
Done.
Searching for "Me, Myself & I" by G-Eazy x Bebe Rexha...
Done.
Searching for "Ride" by twenty one pilots...
Done.
Searching for "Heathens" by twenty one pilots...
Done.
Searching for "Pillowtalk" by Zayn...
Done.
Searching for "Stitches" by Shawn Mendes...
Done.
Searching for "Hotline Bling" by Drake...
Done.
Searching for "Cold Water" by Major Lazer Featuring Just

Done.
Searching for "Paris" by The Chainsmokers...
Done.
Searching for "Side To Side" by Ariana Grande Featuring Nicki Minaj...
Done.
Searching for "Rockabye" by Clean Bandit Featuring Sean Paul & Anne-Marie...
Done.
Searching for "Feel It Still" by Portugal. The Man...
Done.
Searching for "Let Me Love You" by DJ Snake Featuring Justin Bieber...
Done.
Searching for "Sorry Not Sorry" by Demi Lovato...
Done.
Searching for "Bank Account" by 21 Savage...
Done.
Searching for "Can't Stop The Feeling!" by Justin Timberlake...
Done.
Searching for "Mi Gente" by J Balvin & Willy William Featuring Beyonce...
Done.
Searching for "Thunder" by Imagine Dragons...
Done.
Searching for "T-Shirt" by Migos...
Done.
Searching for "Rake It Up" by Yo Gotti Featuring Nicki Minaj...
Done.
Searching for "Mercy" by Shawn Mendes...
Done.
Searching for "Tunnel Vision" by Kodak Black...
Done.
Searching for "Rockstar" by Post Malone Featuring 21 Savage...
Done.
Searching for "In Case You Didn't Know" by Brett Young.

Done.
Searching for "Happier" by Marshmello & Bastille...
Done.
Searching for "Te Bote" by Casper Magico, Nio Garcia, Darell, Nicky Jam, Ozuna & Bad Bunny...
Done.
Searching for "Simple" by Florida Georgia Line...
Done.
Searching for "Lemon" by N*E*R*D & Rihanna...
Done.
Searching for "1-800-273-8255" by Logic Featuring Alessia Cara & Khalid...
Done.
Searching for "Say Something" by Justin Timberlake Featuring Chris Stapleton...
Done.
Searching for "I'm Upset" by Drake...
Done.
Searching for "Get Along" by Kenny Chesney...
Done.
Searching for "Moonlight" by XXXTENTACION...
Done.
Searching for "What Lovers Do" by Maroon 5 Featuring SZA...
Done.
Searching for "X" by Nicky Jam x J Balvin...
Done.
Searching for "Outside Today" by YoungBoy Never Broke Again...
Done.
Searching for "Trip" by Ella Mai...
Done.
Searching for "Dura" by Daddy Yankee...
Done.
Searching for "Changes" by XXXTENTACION...
Done.
Searching for "Mercy" by Brett Young...
Done.
Searching for "One Number Away" by Luke Combs

Done.
Searching for "Roses" by SAINt JHN...
Done.
Searching for "Watermelon Sugar" by Harry Styles...
Done.
Searching for "Before You Go" by Lewis Capaldi...
Done.
Searching for "Falling" by Trevor Daniel...
Done.
Searching for "10,000 Hours" by Dan + Shay & Justin Bieber...
Done.
Searching for "WAP" by Cardi B Featuring Megan Thee Stallion...
Done.
Searching for "Ballin" by Mustard & Roddy Ricch...
Done.
Searching for "Hot Girl Bummer" by blackbear...
Done.
Searching for "Blueberry Faygo" by Lil Mosey...
Done.
Searching for "Heartless" by The Weeknd...
Done.
Searching for "BOP" by DaBaby...
Done.
Searching for "Lose You To Love Me" by Selena Gomez...
Done.
Searching for "Good As Hell" by Lizzo...
Done.
Searching for "Toosie Slide" by Drake...
Done.
Searching for "Break My Heart" by Dua Lipa...
Done.
Searching for "Chasin' You" by Morgan Wallen...
Done.
Searching for "Savage Love (Laxed - Siren Beat)" by Jawsh 685 x Jason Derulo...
Done.
Searching for "No Guidance" by Chris Brown Featu

Done.
Searching for "Body" by Megan Thee Stallion...
Done.
Searching for "Willow" by Taylor Swift...
Done.
Searching for "Bang!" by AJR...
Done.
Searching for "Better Together" by Luke Combs...
Done.
Searching for "You're Mines Still" by Yung Bleu Featuring Drake...
Done.
Searching for "Every Chance I Get" by DJ Khaled Featuring Lil Baby & Lil Durk...
Done.
Searching for "Essence" by Wizkid Featuring Justin Bieber & Tems...
Done.
Searching for "Chasing After You" by Ryan Hurd With Maren Morris...
Done.
Searching for "The Good Ones" by Gabby Barrett...
Done.
Searching for "Leave Before You Love Me" by Marshmello X Jonas Brothers...
Done.
Searching for "Glad You Exist" by Dan + Shay...
Done.
Searching for "Lonely" by Justin Bieber & benny blanco...
Done.
Searching for "Beggin" by Maneskin...
Done.
Searching for "Streets" by Doja Cat...
Done.
Searching for "What's Next" by Drake...
Done.
Searching for "Famous Friends" by Chris Young + Kane Brown...
Done.
Searching for "Lil Bit" by Nelly &

### Detecting language of lyrics

In [11]:
# Language detection
def language_detect(json_name):
    with open(jsons_path + json_name, 'r', encoding = 'UTF-8') as f:
        data = json.load(f)
        try:
            language = detect(data['lyrics'])
        except:
            language = 'error'
    return language
all_time_billboard_wrap_up['Song Language'] = [language_detect(json_name) for json_name in all_time_billboard_wrap_up['JSON file']]
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year,JSON file,Split Names,Song Language
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,0.json,Gordon Jenkins,error
1,2,Nat King Cole,Mona Lisa,1950,1.json,Nat King Cole,en
2,3,Anton Karas,Third Man Theme,1950,2.json,Anton Karas,en
3,4,Gary and Bing Crosby,Sam's Song,1950,3.json,Gary,en
4,5,Gary and Bing Crosby,Simple Melody,1950,4.json,Gary,en
...,...,...,...,...,...,...,...
6775,96,Lainey Wilson,Things A Man Oughta Know,2021,6775.json,Lainey Wilson,en
6776,97,BRS Kash,Throat Baby (Go Baby),2021,6776.json,BRS Kash,en
6777,98,Rod Wave,Tombstone,2021,6777.json,Rod Wave,en
6778,99,Chase Rice Featuring Florida Georgia Line,Drinkin' Beer. Talkin' God. Amen.,2021,6778.json,Chase Rice,en


### Spotify data - IDs, Songs, Artists and Genres

In [12]:
# Create copy of dataframe to manipulate Spotify data
spotify_data = all_time_billboard_wrap_up[['Position', 'Artist', 'Song Title', 'Year', 'JSON file', 'Split Names']].copy()

In [13]:
# Function returns Spotify ID, Song Title, Artist and Genre
def get_spotify_data(artist, track):
    sp_id_list = []
    music_genre_list = []
    audio_features = {}
    try:
        track_id = sp.search(q = 'artist:' + artist + ' track:' + track, type = 'track')
        sp_id = track_id['tracks']['items'][0]['id']
    except IndexError:
        sp_id = ''
    if sp_id != '':
        track = sp.track(sp_id)
        sp_song_name = track['name']
        sp_artist = track['album']['artists'][0].get('name')
        sp_music_genre = sp.artist(track['album']['artists'][0]['uri'])['genres']
        return sp_id, sp_artist, sp_song_name, sp_music_genre

In [14]:
# Populating lists of Spotify data
spotify_ids = []
spotify_artists = []
spotify_songs = []
spotify_music_genres = []
for i in spotify_data.index:
    data = get_spotify_data(spotify_data['Split Names'][i], spotify_data['Song Title'][i])
    if data is None:
        data = '', '', '', []
    spotify_ids.append(data[0])
    spotify_artists.append(data[1])
    spotify_songs.append(data[2])
    spotify_music_genres.append(data[3])

In [15]:
# Assigning data to new columns
spotify_data['Spotify ID'] = spotify_ids
spotify_data['Spotify Artist'] = spotify_artists
spotify_data['Spotify Song Title'] = spotify_songs
spotify_data['Spotify Genre'] = spotify_music_genres
display(spotify_data)

,Position,Artist,Song Title,Year,JSON file,Split Names,Spotify ID,Spotify Artist,Spotify Song Title,Spotify Genre
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,0.json,Gordon Jenkins,1fhLgOJgIIZEsWWffk8ljs,The Weavers,Goodnight Irene,"[american folk revival, folk]"
1,2,Nat King Cole,Mona Lisa,1950,1.json,Nat King Cole,5dae01pKNjRQtgOeAkFzPY,Nat King Cole,Mona Lisa,"[adult standards, lounge, vocal jazz]"
2,3,Anton Karas,Third Man Theme,1950,2.json,Anton Karas,1jWbae7ESwl2fWHGwcLvLk,Anton Karas,The Third Man Theme ( Studio Version),"[classic soundtrack, zither]"
3,4,Gary and Bing Crosby,Sam's Song,1950,3.json,Gary,2KwMacIskAerTgWCxqcuVy,"Mr. I, Gary Q & The Rainbow Singers",Sambalale - Fun Brazillian Folk Song,[]
4,5,Gary and Bing Crosby,Simple Melody,1950,4.json,Gary,33FdwvfiSBRbE2AQ4UehaR,Bing Crosby,Play A Simple Melody - Single Version,"[adult standards, easy listening, lounge]"
...,...,...,...,...,...,...,...,...,...,...
6775,96,Lainey Wilson,Things A Man Oughta Know,2021,6775.json,Lainey Wilson,5QS8PNEWbqTEZyQ6e9ZbJf,Lainey Wilson,Things A Man Oughta Know,"[contemporary country, country pop]"
6776,97,BRS Kash,Throat Baby (Go Baby),2021,6776.json,BRS Kash,15C4TnrrVdym7UykxQIOTZ,BRS Kash,Throat Baby (Go Baby) (with DaBaby & City Girl...,[]
6777,98,Rod Wave,Tombstone,2021,6777.json,Rod Wave,3zc8VZEpM1onYV4FWGdFvm,Rod Wave,Tombstone,[florida rap]
6778,99,Chase Rice Featuring Florida Georgia Line,Drinkin' Beer. Talkin' God. Amen.,2021,6778.json,Chase Rice,1UYfAU2bwgjaM5rIIPQleC,Chase Rice,Drinkin' Beer. Talkin' God. Amen. (feat. Flori...,"[contemporary country, country, country road, ..."


In [16]:
sum(spotify_data['Spotify ID'] == '')

616

In [17]:
# Removing empty Spotify IDs rows and filtering data by Artist and Spotify Artist inclusion
spotify_data = spotify_data[spotify_data['Spotify ID'] != '']
spotify_data = spotify_data[spotify_data['Spotify Artist'].isin(spotify_data['Artist'])]
display(spotify_data)

,Position,Artist,Song Title,Year,JSON file,Split Names,Spotify ID,Spotify Artist,Spotify Song Title,Spotify Genre
1,2,Nat King Cole,Mona Lisa,1950,1.json,Nat King Cole,5dae01pKNjRQtgOeAkFzPY,Nat King Cole,Mona Lisa,"[adult standards, lounge, vocal jazz]"
2,3,Anton Karas,Third Man Theme,1950,2.json,Anton Karas,1jWbae7ESwl2fWHGwcLvLk,Anton Karas,The Third Man Theme ( Studio Version),"[classic soundtrack, zither]"
4,5,Gary and Bing Crosby,Simple Melody,1950,4.json,Gary,33FdwvfiSBRbE2AQ4UehaR,Bing Crosby,Play A Simple Melody - Single Version,"[adult standards, easy listening, lounge]"
5,6,Teresa Brewer,"Music, Music, Music",1950,5.json,Teresa Brewer,0lO5EKoz1Rb1pJoPoldE4D,Teresa Brewer,(Put Another Nickel In) Music! Music! Music!,"[adult standards, brill building pop, deep adu..."
6,7,Guy Lombardo,Third Man Theme,1950,6.json,Guy Lombardo,6kVfwQOc2PjVPwbk0aEmN9,Guy Lombardo,Third Man Theme,"[adult standards, big band, easy listening, lo..."
...,...,...,...,...,...,...,...,...,...,...
6775,96,Lainey Wilson,Things A Man Oughta Know,2021,6775.json,Lainey Wilson,5QS8PNEWbqTEZyQ6e9ZbJf,Lainey Wilson,Things A Man Oughta Know,"[contemporary country, country pop]"
6776,97,BRS Kash,Throat Baby (Go Baby),2021,6776.json,BRS Kash,15C4TnrrVdym7UykxQIOTZ,BRS Kash,Throat Baby (Go Baby) (with DaBaby & City Girl...,[]
6777,98,Rod Wave,Tombstone,2021,6777.json,Rod Wave,3zc8VZEpM1onYV4FWGdFvm,Rod Wave,Tombstone,[florida rap]
6778,99,Chase Rice Featuring Florida Georgia Line,Drinkin' Beer. Talkin' God. Amen.,2021,6778.json,Chase Rice,1UYfAU2bwgjaM5rIIPQleC,Chase Rice,Drinkin' Beer. Talkin' God. Amen. (feat. Flori...,"[contemporary country, country, country road, ..."


### Spotify audio features

In [18]:
# Function returns audio features for specific Spotify ID
def get_audio_features(spotify_id):
    af = pd.DataFrame(sp.audio_features(spotify_id))
    return af

temp_audio_features_list = []
for i in spotify_data['Spotify ID']:
    temp_song_audio_features = get_audio_features(i)
    temp_audio_features_list.append(temp_song_audio_features)

temp_audio_features_df = pd.concat(temp_audio_features_list)

In [23]:
# Merge spotify_data with Audio Features
spotify_data = spotify_data.merge(temp_audio_features_df, how = 'inner', left_on = 'Spotify ID', right_on = 'id')
display(spotify_data)

,Position,Artist,Song Title,Year,JSON file,Split Names,Spotify ID,Spotify Artist,Spotify Song Title,Spotify Genre,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,2,Nat King Cole,Mona Lisa,1950,1.json,Nat King Cole,5dae01pKNjRQtgOeAkFzPY,Nat King Cole,Mona Lisa,"[adult standards, lounge, vocal jazz]",...,0.1130,-12.201,1.0,0.0371,174.578,3.0,https://api.spotify.com/v1/tracks/5dae01pKNjRQ...,audio_features,spotify:track:5dae01pKNjRQtgOeAkFzPY,0.303
1,3,Anton Karas,Third Man Theme,1950,2.json,Anton Karas,1jWbae7ESwl2fWHGwcLvLk,Anton Karas,The Third Man Theme ( Studio Version),"[classic soundtrack, zither]",...,0.0645,-8.929,1.0,0.0413,87.948,4.0,https://api.spotify.com/v1/tracks/1jWbae7ESwl2...,audio_features,spotify:track:1jWbae7ESwl2fWHGwcLvLk,0.835
2,5,Gary and Bing Crosby,Simple Melody,1950,4.json,Gary,33FdwvfiSBRbE2AQ4UehaR,Bing Crosby,Play A Simple Melody - Single Version,"[adult standards, easy listening, lounge]",...,0.1670,-13.755,1.0,0.0473,138.800,3.0,https://api.spotify.com/v1/tracks/33FdwvfiSBRb...,audio_features,spotify:track:33FdwvfiSBRbE2AQ4UehaR,0.829
3,6,Teresa Brewer,"Music, Music, Music",1950,5.json,Teresa Brewer,0lO5EKoz1Rb1pJoPoldE4D,Teresa Brewer,(Put Another Nickel In) Music! Music! Music!,"[adult standards, brill building pop, deep adu...",...,0.1540,-14.392,1.0,0.0398,99.136,4.0,https://api.spotify.com/v1/tracks/0lO5EKoz1Rb1...,audio_features,spotify:track:0lO5EKoz1Rb1pJoPoldE4D,0.919
4,7,Guy Lombardo,Third Man Theme,1950,6.json,Guy Lombardo,6kVfwQOc2PjVPwbk0aEmN9,Guy Lombardo,Third Man Theme,"[adult standards, big band, easy listening, lo...",...,0.0849,-10.808,1.0,0.0358,115.590,4.0,https://api.spotify.com/v1/tracks/6kVfwQOc2PjV...,audio_features,spotify:track:6kVfwQOc2PjVPwbk0aEmN9,0.536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4932,96,Lainey Wilson,Things A Man Oughta Know,2021,6775.json,Lainey Wilson,5QS8PNEWbqTEZyQ6e9ZbJf,Lainey Wilson,Things A Man Oughta Know,"[contemporary country, country pop]",...,0.1330,-5.623,1.0,0.0312,139.931,4.0,https://api.spotify.com/v1/tracks/5QS8PNEWbqTE...,audio_features,spotify:track:5QS8PNEWbqTEZyQ6e9ZbJf,0.397
4933,97,BRS Kash,Throat Baby (Go Baby),2021,6776.json,BRS Kash,15C4TnrrVdym7UykxQIOTZ,BRS Kash,Throat Baby (Go Baby) (with DaBaby & City Girl...,[],...,0.1670,-8.420,0.0,0.3240,131.988,4.0,https://api.spotify.com/v1/tracks/15C4TnrrVdym...,audio_features,spotify:track:15C4TnrrVdym7UykxQIOTZ,0.397
4934,98,Rod Wave,Tombstone,2021,6777.json,Rod Wave,3zc8VZEpM1onYV4FWGdFvm,Rod Wave,Tombstone,[florida rap],...,0.1290,-5.212,1.0,0.1630,84.448,4.0,https://api.spotify.com/v1/tracks/3zc8VZEpM1on...,audio_features,spotify:track:3zc8VZEpM1onYV4FWGdFvm,0.535
4935,99,Chase Rice Featuring Florida Georgia Line,Drinkin' Beer. Talkin' God. Amen.,2021,6778.json,Chase Rice,1UYfAU2bwgjaM5rIIPQleC,Chase Rice,Drinkin' Beer. Talkin' God. Amen. (feat. Flori...,"[contemporary country, country, country road, ...",...,0.3740,-4.691,1.0,0.0294,100.032,4.0,https://api.spotify.com/v1/tracks/1UYfAU2bwgja...,audio_features,spotify:track:1UYfAU2bwgjaM5rIIPQleC,0.724


In [24]:
# Drop duplicates
spotify_data = spotify_data.loc[spotify_data.astype(str).drop_duplicates().index]
display(spotify_data)

,Position,Artist,Song Title,Year,JSON file,Split Names,Spotify ID,Spotify Artist,Spotify Song Title,Spotify Genre,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,2,Nat King Cole,Mona Lisa,1950,1.json,Nat King Cole,5dae01pKNjRQtgOeAkFzPY,Nat King Cole,Mona Lisa,"[adult standards, lounge, vocal jazz]",...,0.1130,-12.201,1.0,0.0371,174.578,3.0,https://api.spotify.com/v1/tracks/5dae01pKNjRQ...,audio_features,spotify:track:5dae01pKNjRQtgOeAkFzPY,0.303
1,3,Anton Karas,Third Man Theme,1950,2.json,Anton Karas,1jWbae7ESwl2fWHGwcLvLk,Anton Karas,The Third Man Theme ( Studio Version),"[classic soundtrack, zither]",...,0.0645,-8.929,1.0,0.0413,87.948,4.0,https://api.spotify.com/v1/tracks/1jWbae7ESwl2...,audio_features,spotify:track:1jWbae7ESwl2fWHGwcLvLk,0.835
2,5,Gary and Bing Crosby,Simple Melody,1950,4.json,Gary,33FdwvfiSBRbE2AQ4UehaR,Bing Crosby,Play A Simple Melody - Single Version,"[adult standards, easy listening, lounge]",...,0.1670,-13.755,1.0,0.0473,138.800,3.0,https://api.spotify.com/v1/tracks/33FdwvfiSBRb...,audio_features,spotify:track:33FdwvfiSBRbE2AQ4UehaR,0.829
3,6,Teresa Brewer,"Music, Music, Music",1950,5.json,Teresa Brewer,0lO5EKoz1Rb1pJoPoldE4D,Teresa Brewer,(Put Another Nickel In) Music! Music! Music!,"[adult standards, brill building pop, deep adu...",...,0.1540,-14.392,1.0,0.0398,99.136,4.0,https://api.spotify.com/v1/tracks/0lO5EKoz1Rb1...,audio_features,spotify:track:0lO5EKoz1Rb1pJoPoldE4D,0.919
4,7,Guy Lombardo,Third Man Theme,1950,6.json,Guy Lombardo,6kVfwQOc2PjVPwbk0aEmN9,Guy Lombardo,Third Man Theme,"[adult standards, big band, easy listening, lo...",...,0.0849,-10.808,1.0,0.0358,115.590,4.0,https://api.spotify.com/v1/tracks/6kVfwQOc2PjV...,audio_features,spotify:track:6kVfwQOc2PjVPwbk0aEmN9,0.536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4932,96,Lainey Wilson,Things A Man Oughta Know,2021,6775.json,Lainey Wilson,5QS8PNEWbqTEZyQ6e9ZbJf,Lainey Wilson,Things A Man Oughta Know,"[contemporary country, country pop]",...,0.1330,-5.623,1.0,0.0312,139.931,4.0,https://api.spotify.com/v1/tracks/5QS8PNEWbqTE...,audio_features,spotify:track:5QS8PNEWbqTEZyQ6e9ZbJf,0.397
4933,97,BRS Kash,Throat Baby (Go Baby),2021,6776.json,BRS Kash,15C4TnrrVdym7UykxQIOTZ,BRS Kash,Throat Baby (Go Baby) (with DaBaby & City Girl...,[],...,0.1670,-8.420,0.0,0.3240,131.988,4.0,https://api.spotify.com/v1/tracks/15C4TnrrVdym...,audio_features,spotify:track:15C4TnrrVdym7UykxQIOTZ,0.397
4934,98,Rod Wave,Tombstone,2021,6777.json,Rod Wave,3zc8VZEpM1onYV4FWGdFvm,Rod Wave,Tombstone,[florida rap],...,0.1290,-5.212,1.0,0.1630,84.448,4.0,https://api.spotify.com/v1/tracks/3zc8VZEpM1on...,audio_features,spotify:track:3zc8VZEpM1onYV4FWGdFvm,0.535
4935,99,Chase Rice Featuring Florida Georgia Line,Drinkin' Beer. Talkin' God. Amen.,2021,6778.json,Chase Rice,1UYfAU2bwgjaM5rIIPQleC,Chase Rice,Drinkin' Beer. Talkin' God. Amen. (feat. Flori...,"[contemporary country, country, country road, ...",...,0.3740,-4.691,1.0,0.0294,100.032,4.0,https://api.spotify.com/v1/tracks/1UYfAU2bwgja...,audio_features,spotify:track:1UYfAU2bwgjaM5rIIPQleC,0.724


In [26]:
# Cleaning and saving spotify_data dataframe
spotify_data = spotify_data.drop(spotify_data.columns[10], axis = 1)
spotify_data.columns = spotify_data.columns.str.rstrip('_x')
display(spotify_data)
spotify_data.to_csv(wd + '\\spotify_data.csv', index = False)

,Position,Artist,Song Title,Year,JSON file,Split Names,Spotify ID,Spotify Artist,Spotify Song Title,Spotify Genre,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,2,Nat King Cole,Mona Lisa,1950,1.json,Nat King Cole,5dae01pKNjRQtgOeAkFzPY,Nat King Cole,Mona Lisa,"[adult standards, lounge, vocal jazz]",...,0.1130,-12.201,1.0,0.0371,174.578,3.0,https://api.spotify.com/v1/tracks/5dae01pKNjRQ...,audio_features,spotify:track:5dae01pKNjRQtgOeAkFzPY,0.303
1,3,Anton Karas,Third Man Theme,1950,2.json,Anton Karas,1jWbae7ESwl2fWHGwcLvLk,Anton Karas,The Third Man Theme ( Studio Version),"[classic soundtrack, zither]",...,0.0645,-8.929,1.0,0.0413,87.948,4.0,https://api.spotify.com/v1/tracks/1jWbae7ESwl2...,audio_features,spotify:track:1jWbae7ESwl2fWHGwcLvLk,0.835
2,5,Gary and Bing Crosby,Simple Melody,1950,4.json,Gary,33FdwvfiSBRbE2AQ4UehaR,Bing Crosby,Play A Simple Melody - Single Version,"[adult standards, easy listening, lounge]",...,0.1670,-13.755,1.0,0.0473,138.800,3.0,https://api.spotify.com/v1/tracks/33FdwvfiSBRb...,audio_features,spotify:track:33FdwvfiSBRbE2AQ4UehaR,0.829
3,6,Teresa Brewer,"Music, Music, Music",1950,5.json,Teresa Brewer,0lO5EKoz1Rb1pJoPoldE4D,Teresa Brewer,(Put Another Nickel In) Music! Music! Music!,"[adult standards, brill building pop, deep adu...",...,0.1540,-14.392,1.0,0.0398,99.136,4.0,https://api.spotify.com/v1/tracks/0lO5EKoz1Rb1...,audio_features,spotify:track:0lO5EKoz1Rb1pJoPoldE4D,0.919
4,7,Guy Lombardo,Third Man Theme,1950,6.json,Guy Lombardo,6kVfwQOc2PjVPwbk0aEmN9,Guy Lombardo,Third Man Theme,"[adult standards, big band, easy listening, lo...",...,0.0849,-10.808,1.0,0.0358,115.590,4.0,https://api.spotify.com/v1/tracks/6kVfwQOc2PjV...,audio_features,spotify:track:6kVfwQOc2PjVPwbk0aEmN9,0.536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4932,96,Lainey Wilson,Things A Man Oughta Know,2021,6775.json,Lainey Wilson,5QS8PNEWbqTEZyQ6e9ZbJf,Lainey Wilson,Things A Man Oughta Know,"[contemporary country, country pop]",...,0.1330,-5.623,1.0,0.0312,139.931,4.0,https://api.spotify.com/v1/tracks/5QS8PNEWbqTE...,audio_features,spotify:track:5QS8PNEWbqTEZyQ6e9ZbJf,0.397
4933,97,BRS Kash,Throat Baby (Go Baby),2021,6776.json,BRS Kash,15C4TnrrVdym7UykxQIOTZ,BRS Kash,Throat Baby (Go Baby) (with DaBaby & City Girl...,[],...,0.1670,-8.420,0.0,0.3240,131.988,4.0,https://api.spotify.com/v1/tracks/15C4TnrrVdym...,audio_features,spotify:track:15C4TnrrVdym7UykxQIOTZ,0.397
4934,98,Rod Wave,Tombstone,2021,6777.json,Rod Wave,3zc8VZEpM1onYV4FWGdFvm,Rod Wave,Tombstone,[florida rap],...,0.1290,-5.212,1.0,0.1630,84.448,4.0,https://api.spotify.com/v1/tracks/3zc8VZEpM1on...,audio_features,spotify:track:3zc8VZEpM1onYV4FWGdFvm,0.535
4935,99,Chase Rice Featuring Florida Georgia Line,Drinkin' Beer. Talkin' God. Amen.,2021,6778.json,Chase Rice,1UYfAU2bwgjaM5rIIPQleC,Chase Rice,Drinkin' Beer. Talkin' God. Amen. (feat. Flori...,"[contemporary country, country, country road, ...",...,0.3740,-4.691,1.0,0.0294,100.032,4.0,https://api.spotify.com/v1/tracks/1UYfAU2bwgja...,audio_features,spotify:track:1UYfAU2bwgjaM5rIIPQleC,0.724


### Deleting incorrect json files

In [28]:
# Deteling json files that are bigger than 5KB
all_json_files = os.listdir(jsons_path)
for x in all_json_files:
    filepath = jsons_path + str(x)
    json_size = os.path.getsize(filepath)
    if (json_size > 5 * 1024):
        os.remove(filepath)
        
# Selecting all_billboard_wrap_up data corresponding with actual list of jsons
ordered_jsons = sorted([int(x[:-5]) for x in os.listdir(jsons_path)])
all_time_billboard_wrap_up_cleaned = all_time_billboard_wrap_up.iloc[ordered_jsons]
display(all_time_billboard_wrap_up_cleaned)

,Position,Artist,Song Title,Year,JSON file,Split Names,Song Language
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,0.json,Gordon Jenkins,error
1,2,Nat King Cole,Mona Lisa,1950,1.json,Nat King Cole,en
3,4,Gary and Bing Crosby,Sam's Song,1950,3.json,Gary,en
4,5,Gary and Bing Crosby,Simple Melody,1950,4.json,Gary,en
5,6,Teresa Brewer,"Music, Music, Music",1950,5.json,Teresa Brewer,en
...,...,...,...,...,...,...,...
6775,96,Lainey Wilson,Things A Man Oughta Know,2021,6775.json,Lainey Wilson,en
6776,97,BRS Kash,Throat Baby (Go Baby),2021,6776.json,BRS Kash,en
6777,98,Rod Wave,Tombstone,2021,6777.json,Rod Wave,en
6778,99,Chase Rice Featuring Florida Georgia Line,Drinkin' Beer. Talkin' God. Amen.,2021,6778.json,Chase Rice,en


In [29]:
# Filtering language data and saving all_time_billboard_wrap_up_cleaned_data to csv
all_time_billboard_wrap_up_cleaned = all_time_billboard_wrap_up_cleaned[all_time_billboard_wrap_up_cleaned['Song Language'].isin(['en'])]
all_time_billboard_wrap_up_cleaned.to_csv(wd + '\\lyrics_metadata.csv', index = False)